# Introduction

In this notebook I implement a basic fine-mapping method.

### Load modules and data

In [8]:
import numpy
from matplotlib import pyplot as plt
import itertools as it

%matplotlib inline

s_tss_1=numpy.load('summary_stats_g1_tss60.npy')[0]
s_tss_2=numpy.load('summary_stats_g2_tss60.npy')[0]
LD_tss_1=numpy.load('LD_g1_TSS60.npy')
LD_tss_2=numpy.load('LD_g2_TSS60.npy')

### Generate z arrays

In [90]:
z1 = numpy.array(numpy.divide(s_tss_1['beta'],numpy.sqrt(s_tss_1['var_beta'])))
z2 = numpy.array(numpy.divide(s_tss_2['beta'],numpy.sqrt(s_tss_1['var_beta'])))
z1 = numpy.ndarray.flatten(z1)
z2 = numpy.ndarray.flatten(z2)

### Create selection of SNPs

In [167]:
def select_snps(z, subset):
    return [z[i] for i in subset]

#example
# for subset in it.combinations(range(len(z1)),3):
#     print subset, select_snps(z1, subset)    

### Select covariance submatrix

In [166]:
def select_cov(cov, subset):
    return cov[numpy.ix_(subset,subset)]

#example   
#select_cov(LD_tss_1, (0,1,5))

### Calculate Bayes Factor

In [193]:
def calc_BF(z, cov, v):
    z = numpy.matrix(z)
    z = z.T
    v = numpy.matrix(v)
    n = len(z)
    coeff = numpy.linalg.det((numpy.matrix(numpy.eye(n)) + n * numpy.matrix(v) * numpy.matrix(cov)).I)
    exponent = 0.5* z.T * numpy.matrix((n*v.I + cov)).I * z
    return numpy.array(coeff * numpy.exp(exponent))[0][0]

# example
# cov = select_cov(LD_tss_1, (0,1,4,7))
# z = select_snps(z1, (0,1,4,7))
# v = numpy.eye(len(z))/1000
# calc_BF(z,cov,v)


    
    

In [220]:
def choose_variant_set(data,k):
    results = []
    for i in range(1,k):
        for subset in it.combinations(range(len(data[0])),i):
            z = select_snps(data[0], subset)
            cov = select_cov(data[1],subset)
            v = numpy.eye(len(z))/1000
            results.append((subset, calc_BF(z, cov,v)))
    results.sort(key=lambda x: x[1], reverse=True)
    return results[0:10]

k=5
data = (z1, LD_tss_1)
data = (z2, LD_tss_2)
    
set1 = choose_variant_set((z1, LD_tss_1),k)
set2 = choose_variant_set((z2, LD_tss_2),k)

In [221]:
set1

[((29,), 1.2946508909185706),
 ((26, 29), 1.1748163653482278),
 ((28, 29), 1.1614149860995089),
 ((29, 32), 1.1599525170197742),
 ((29, 30), 1.1597843152040157),
 ((27, 29), 1.1502598411510934),
 ((29, 33), 1.1455678596163543),
 ((14, 29), 1.1349748313852674),
 ((6, 29), 1.1344972524421446),
 ((29, 31), 1.1344750945696898)]

In [222]:
set2

[((29,), 14.233972108905558),
 ((29, 33), 6.8693712209155571),
 ((26, 29), 5.3803984459592282),
 ((28, 29), 4.8819100892198781),
 ((26, 29, 33), 4.5599390656398695),
 ((29, 30), 4.4862638699929311),
 ((28, 29, 33), 4.2736505828702356),
 ((27, 29), 4.1863270971450364),
 ((29, 31), 4.0627197926249368),
 ((29, 30, 33), 4.0395111813402558)]